In [1]:
import os.path as osp
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from typing import Dict, List, Tuple
from torch.nn import BatchNorm1d
from torchmetrics import Accuracy
from torch_geometric.data.lightning import LightningNodeData
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import Tensor
import torch_geometric.transforms as T
import torch_geometric
from torch_geometric.data import Batch
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import Linear, SAGEConv, to_hetero
from torch_geometric.typing import EdgeType, NodeType

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels: int, out_channels: int,
                 dropout: float):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels)
        self.lin = Linear(-1, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        return self.lin(x)


class Model(LightningModule):
    def __init__(self,
        metadata: Tuple[List[NodeType], List[EdgeType]],
        hidden_channels: int,
        out_channels: int,
        dropout: float):
        super().__init__()
        self.save_hyperparameters()

        model = GNN(hidden_channels, out_channels, dropout)

        # Convert the homogeneous GNN model to a heterogeneous variant in
        # which distinct parameters are learned for each node and edge type.
        self.model = to_hetero(model, metadata, aggr='sum')

        self.train_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.val_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.test_acc = Accuracy(task='multiclass', num_classes=out_channels)

    def forward(
        self,
        x_dict: Dict[NodeType, Tensor],
        edge_index_dict: Dict[EdgeType, Tensor],
    ) -> Dict[NodeType, Tensor]:
        return self.model(x_dict, edge_index_dict)

    def common_step(self, batch: Batch) -> Tuple[Tensor, Tensor]:
        node_type = 'paper'
        batch_size = batch[node_type].batch_size
        y_hat = self(batch.x_dict, batch.edge_index_dict)[node_type][:batch_size]
        y = batch[node_type].y[:batch_size]
        return y_hat, y

    def training_step(self, batch: Batch, batch_idx: int) -> Tensor:
        node_type = 'paper'
        y_hat, y = self.common_step(batch)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat.softmax(dim=-1), y)
        self.log('train_acc', self.train_acc, prog_bar=True, on_step=False,
                 on_epoch=True)
        return loss

    def validation_step(self, batch: Batch, batch_idx: int):
        node_type = 'paper'
        y_hat, y = self.common_step(batch)
        self.val_acc(y_hat.softmax(dim=-1), y)
        self.log('val_acc', self.val_acc, prog_bar=True, on_step=False,
                 on_epoch=True)

    def test_step(self, batch: Batch, batch_idx: int):
        node_type = 'paper'
        y_hat, y = self.common_step(batch)
        self.test_acc(y_hat.softmax(dim=-1), y)
        self.log('test_acc', self.test_acc, prog_bar=True, on_step=False,
                 on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)
    
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")

PyTorch version: 2.6.0+cu126
PyTorch Geometric version: 2.7.0
PyTorch Lightning version: 2.5.0.post0


In [ ]:
torch.set_float32_matmul_precision('medium' or 'high')
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
dataset = OGB_MAG(osp.join('data', 'OGB'), preprocess='metapath2vec', transform=T.ToUndirected(merge=False))
data = dataset[0]

node_type = 'paper'

datamodule = LightningNodeData(
    data,
    input_train_nodes=(node_type, data[node_type].train_mask),
    input_val_nodes=(node_type, data[node_type].val_mask),
    input_test_nodes=(node_type, data[node_type].test_mask),
    loader='neighbor',
    num_neighbors=[10, 10],
    batch_size=32,
    num_workers=0,
)

model = Model(data.metadata(), hidden_channels=64,
                        out_channels=349, dropout=0.0)

with torch.no_grad():  # Run a dummy forward pass to initialize lazy model
    loader = datamodule.train_dataloader()
    batch = next(iter(loader))
    model.common_step(batch)

strategy = pl.strategies.SingleDeviceStrategy('cuda:0')
checkpoint = ModelCheckpoint(monitor='val_acc', save_top_k=1, mode='max')
trainer = Trainer(strategy=strategy, devices=1, max_epochs=20,
                    log_every_n_steps=5, callbacks=[checkpoint])

trainer.fit(model, datamodule)
trainer.test(ckpt_path='best', datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\subud\GitHub Projects\Graph Training\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | GraphModule        | 288 K  | train
1 | train_acc | MulticlassAccuracy | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | Multi

Sanity Checking:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\subud\GitHub Projects\Graph Training\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\subud\GitHub Projects\Graph Training\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0:  18%|█▊        | 3484/19675 [07:02<32:41,  8.25it/s, v_num=5]

In [20]:
!pip3 install setuptools wheel
# Install PyTorch Geometric and related libraries
!pip3 install torch-sparse

^C


In [25]:
!pip3 install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
  Using cached torch_scatter-2.1.2-cp312-cp312-win_amd64.whl
